#### 【 앙상블 – GBM (Gradient Boosting Machine) 】

- 이전 모델의 예측 오차를 손실함수의 기울기(gradient)로 계산 -> 다음 모델 학습하는 알고리즘
- 손실함수 최소화하는 경사하강법 -> 각 단계에서 손실함수 기울기(잔차) 예측하는 결정트리 하나씩 추가
- 회귀 / 분류 모두 사용 가능
- 미분 가능한 손실함수를 단계적으로 최소화하는 알고리즘 => 예: MSE, Log-loss, Huber loss 등

- 각 단계에서 손실함수의 음의 기울기(잔차, residual)를 새로운 모델이 학습
    * 예측이 많이 틀린 샘플 → 큰 잔차(gradient)
    * 예측이 적게 틀린 샘플 → 작은 잔차

[1] Gradient Boosting = 손실함수 기울기(=잔차) 타깃으로 트리 하나씩 더해가는 과정<hr>

- loss를 예측값에 대해 미분해서 나온 gradient를 트리가 근사
- 예시 
    * 구성 : 회귀 + MSE 손실
    * gradient : (y - y_pred) 형태의 잔차로 떨어져 직관이 가장 
    * 트리 : 미분이 아니라 “잔차를 맞추는 회귀” 진행 

In [1]:
## ==================================================
## 모듈 로딩
## ==================================================
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

## 재현성 설정
np.random.seed(42)

## ==================================================
## 1) 장난감 데이터(비선형) 생성
## ==================================================
n = 80
X = np.linspace(-3, 3, n).reshape(-1, 1)
y = (X[:, 0] ** 2) + 0.3 * np.sin(3 * X[:, 0]) + np.random.normal(0, 0.3, size=n)

## ==================================================
## 2) "미니 Gradient Boosting" 구현 (MSE 기준)
##    - F0(x) = y의 평균 (상수 모델)
##    - 매 반복마다:
##        (a) 현재 예측 y_pred = F_{m-1}(x)
##        (b) loss의 기울기: dL/dy_pred = (y_pred - y)
##        (c) 음의 기울기(=pseudo-residual): r = -(dL/dy_pred) = (y - y_pred)
##        (d) 트리 h_m(x)를 r를 타깃으로 학습
##        (e) F_m(x) = F_{m-1}(x) + lr * h_m(x)
## ==================================================
M = 10
lr = 0.3

## 초기 모델: 상수 (y 평균)
F0 = np.mean(y)
y_pred = np.full_like(y, F0, dtype=float)

trees = []

print("== Gradient Boosting 원리 데모 (MSE) ==")
print(f"F0 (상수 예측) = mean(y) = {F0:.4f}")
print(f"[iter 0] MSE = {mean_squared_error(y, y_pred):.6f}\n")

for m in range(1, M + 1):
    # (b) MSE Loss: L = 1/2 * (y - y_pred)^2 라고 두면
    #     dL/dy_pred = (y_pred - y)
    grad = (y_pred - y)

    # (c) 음의 기울기(= pseudo-residual)
    residual = -grad  # == (y - y_pred)

    # (d) residual을 예측하는 약한 학습기(얕은 트리) 학습
    tree = DecisionTreeRegressor(max_depth=2, random_state=42)
    tree.fit(X, residual)
    trees.append(tree)

    # (e) 모델 업데이트
    update = tree.predict(X)
    y_pred = y_pred + lr * update

    mse = mean_squared_error(y, y_pred)

    # 학습 과정 출력(일부 값만)
    if m <= 3:
        print(f"[iter {m}]")
        print(f"  residual(처음 5개) = {residual[:5]}")
        print(f"  tree_update(처음 5개) = {update[:5]}")
        print(f"  MSE = {mse:.6f}\n")
    else:
        print(f"[iter {m}] MSE = {mse:.6f}")

print("\n핵심 요약:")
print(" - 트리를 미분한 게 아니라, Loss를 y_pred에 대해 미분(grad)해서 residual을 만들었고")
print(" - 트리는 그 residual을 '회귀 타깃'으로 근사했으며")
print(" - 모델은 F <- F + lr * tree_update 로 누적 갱신됩니다.")


== Gradient Boosting 원리 데모 (MSE) ==
F0 (상수 예측) = mean(y) = 3.0388
[iter 0] MSE = 7.881530

[iter 1]
  residual(처음 5개) = [5.98658138 5.28761303 5.03587779 4.83445236 3.86868435]
  tree_update(처음 5개) = [3.40571662 3.40571662 3.40571662 3.40571662 3.40571662]
  MSE = 4.644033

[iter 2]
  residual(처음 5개) = [4.96486639 4.26589804 4.0141628  3.81273738 2.84696936]
  tree_update(처음 5개) = [1.69176059 1.69176059 1.69176059 1.69176059 1.69176059]
  MSE = 2.794287

[iter 3]
  residual(처음 5개) = [4.45733821 3.75836986 3.50663462 3.3052092  2.33944118]
  tree_update(처음 5개) = [3.75688797 3.75688797 3.75688797 3.75688797 1.95952193]
  MSE = 1.774396

[iter 4] MSE = 1.206327
[iter 5] MSE = 0.786906
[iter 6] MSE = 0.510643
[iter 7] MSE = 0.359739
[iter 8] MSE = 0.265214
[iter 9] MSE = 0.181347
[iter 10] MSE = 0.142557

핵심 요약:
 - 트리를 미분한 게 아니라, Loss를 y_pred에 대해 미분(grad)해서 residual을 만들었고
 - 트리는 그 residual을 '회귀 타깃'으로 근사했으며
 - 모델은 F <- F + lr * tree_update 로 누적 갱신됩니다.


[2] 회귀 예제 <hr>

In [2]:
## 모듈 로딩
from xgboost import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## 1. 데이터 생성
X, y = make_regression(
    n_samples=1000,
    n_features=10,
    noise=10.0,
    random_state=42
)

## 2. 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## 3. 모델 생성 (CPU)
model = XGBRegressor(
    n_estimators=200,
    max_depth=4,
    learning_rate=0.05,
    tree_method="hist",   # CPU
    random_state=42
)

## 4. 학습
model.fit(X_train, y_train)

## 5. 예측 및 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"MSE: {mse:.2f}")


MSE: 1344.70
